In [1]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import seaborn as sns
import scipy.stats as stats
from scipy.stats import gamma
from scipy.stats import ortho_group

import cupy as cp
import cupyx.scipy 

from Test.hsic_naive import IndpTest_naive
from Test.hsic_lkgau import IndpTest_LKGaussian
from Test.hsic_lklap import IndpTest_LKLaplace
from Test.hsic_lkwgau import IndpTest_LKWeightGaussian
from Test.hsic_lkwlap import IndpTest_LKWeightLaplace
from Test.hsic_lkselect import IndpTest_LKSelect_GauLap
from Test.hsic_kselect import IndpTest_KSelect

device = torch.device('cuda:0')

In [2]:
def generate_ISA(n,d,sigma_normal,alpha):
    
    x = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    y = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    p = np.random.permutation(n)
    y_p = y[p]

    D = np.zeros([2,n])
    D[0,:] = x
    D[1,:] = y_p

    theta = np.pi/4*alpha
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))

    D_R = R@D
    X_mix = D_R[0,:].reshape(-1,1)
    Y_mix = D_R[1,:].reshape(-1,1)

    X_z = np.random.randn(n,d-1)
    Y_z = np.random.randn(n,d-1)

    X_con = np.concatenate((X_mix,X_z), axis=1)
    Y_con = np.concatenate((Y_mix,Y_z), axis=1)

    m_x = ortho_group.rvs(dim=d)
    m_y = ortho_group.rvs(dim=d)

    X = (m_x@X_con.T).T
    Y = (m_y@Y_con.T).T
    
    return X,Y

def Sinusoid(x, y, w):
    return 1 + np.sin(w*x)*np.sin(w*y)

def Sinusoid_Generator(n,w):
    i = 0
    output = np.zeros([n,2])
    while i < n:
        U = np.random.rand(1)
        V = np.random.rand(2)
        x0 = -np.pi + V[0]*2*np.pi
        x1 = -np.pi + V[1]*2*np.pi
        if U < 1/2 * Sinusoid(x0,x1,w):
            output[i, 0] = x0
            output[i, 1] = x1
            i = i + 1
    return output

# n = 1000
# d = 3
def sinedependence(n,d):
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    Z = np.random.randn(n)
    Y = 20*np.sin(4*np.pi*(X[:,0]**2 + X[:,1]**2))+Z 
    return X,Y

# n = 1000
# d = 4
def GSign(n,d):
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    sign_X = np.sign(X)
    Z = np.random.randn(n)
    Y = np.abs(Z)*np.prod(sign_X,1)
    return X,Y

In [3]:
n = 900
d = 3
test_num = 50

result_test_correct = np.zeros([3,test_num])

for it in range(test_num):
    X, Y = sinedependence(n, d)
    Y = Y.reshape(-1,1)
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    #test#
    hsic0 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Gaussian", null_gamma = True)
    results_all0 = hsic0.perform_test()
    result_test_correct[0, it] = float(results_all0['h0_rejected'])

    hsic1 = IndpTest_LKGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
    results_all1 = hsic1.perform_test(debug = -1, if_grid_search = True)
    result_test_correct[1, it] = float(results_all1['h0_rejected'])

    hsic2 = IndpTest_LKWeightGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
    results_all2 = hsic2.perform_test(debug = -1, if_grid_search = True)
    result_test_correct[2, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

0 [1. 1. 0.]
1 [1.  0.5 0.5]
2 [0.66666667 0.33333333 0.66666667]
3 [0.75 0.25 0.75]
4 [0.6 0.4 0.8]
5 [0.66666667 0.5        0.83333333]
6 [0.57142857 0.57142857 0.85714286]
7 [0.5   0.625 0.875]
8 [0.44444444 0.55555556 0.88888889]
9 [0.4 0.6 0.9]
10 [0.45454545 0.63636364 0.90909091]
11 [0.41666667 0.66666667 0.91666667]
12 [0.38461538 0.69230769 0.92307692]
13 [0.42857143 0.71428571 0.92857143]
14 [0.4        0.73333333 0.93333333]
15 [0.4375 0.75   0.9375]
16 [0.47058824 0.70588235 0.94117647]
17 [0.44444444 0.72222222 0.88888889]
18 [0.47368421 0.73684211 0.89473684]
19 [0.45 0.7  0.9 ]
20 [0.42857143 0.71428571 0.9047619 ]
21 [0.40909091 0.72727273 0.90909091]
22 [0.39130435 0.69565217 0.86956522]
23 [0.375      0.70833333 0.875     ]
24 [0.36 0.72 0.88]
25 [0.34615385 0.69230769 0.88461538]
26 [0.33333333 0.66666667 0.88888889]
27 [0.32142857 0.64285714 0.89285714]
28 [0.31034483 0.65517241 0.89655172]
29 [0.33333333 0.66666667 0.9       ]
30 [0.35483871 0.67741935 0.90322581]


In [5]:
n = 400
d = 4
test_num = 50

result_test_correct = np.zeros([3,test_num])

for it in range(test_num):
    X, Y = GSign(n,d)
    Y = Y.reshape(-1,1)
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    #test#
    hsic0 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Gaussian", null_gamma = True)
    results_all0 = hsic0.perform_test()
    result_test_correct[0, it] = float(results_all0['h0_rejected'])

    hsic1 = IndpTest_LKGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
    results_all1 = hsic1.perform_test(debug = -1, if_grid_search = True)
    result_test_correct[1, it] = float(results_all1['h0_rejected'])

    hsic2 = IndpTest_LKWeightGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
    results_all2 = hsic2.perform_test(debug = -1, if_grid_search = True)
    result_test_correct[2, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

0 [0. 0. 0.]
1 [0.  0.5 0. ]
2 [0.         0.66666667 0.        ]
3 [0.   0.75 0.  ]
4 [0.  0.6 0. ]
5 [0.16666667 0.66666667 0.16666667]
6 [0.14285714 0.71428571 0.28571429]
7 [0.125 0.75  0.375]
8 [0.11111111 0.66666667 0.33333333]
9 [0.2 0.6 0.3]
10 [0.18181818 0.54545455 0.27272727]
11 [0.16666667 0.58333333 0.25      ]
12 [0.23076923 0.61538462 0.30769231]
13 [0.21428571 0.64285714 0.35714286]
14 [0.2        0.6        0.33333333]
15 [0.1875 0.625  0.375 ]
16 [0.17647059 0.64705882 0.35294118]
17 [0.22222222 0.66666667 0.33333333]
18 [0.21052632 0.63157895 0.36842105]
19 [0.2 0.6 0.4]
20 [0.19047619 0.61904762 0.38095238]
21 [0.22727273 0.59090909 0.36363636]
22 [0.2173913  0.60869565 0.39130435]
23 [0.20833333 0.625      0.41666667]
24 [0.24 0.64 0.4 ]
25 [0.23076923 0.61538462 0.38461538]
26 [0.22222222 0.62962963 0.37037037]
27 [0.21428571 0.60714286 0.35714286]
28 [0.20689655 0.62068966 0.34482759]
29 [0.23333333 0.63333333 0.36666667]
30 [0.22580645 0.64516129 0.38709677]
31 

In [6]:
# ica experiments #

test_num = 100
result_all_correct = []

# alpha_set = np.linspace(0.0, 1.0, num=10)
alpha_set = [0.7]

confident_level = 0.05

for alpha in alpha_set:
    result_test_correct = np.zeros([3,test_num])
    
    for it in range(test_num):
        n = 128
        d = 4
        sigma_normal = 0.1
        X, Y = generate_ISA(n,d,sigma_normal,alpha)
        X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
        
        #test#
        hsic0 = IndpTest_naive(X_tensor, Y_tensor, alpha=0.05, n_permutation=100, kernel_type="Gaussian", null_gamma = True)
        results_all0 = hsic0.perform_test()
        result_test_correct[0, it] = float(results_all0['h0_rejected'])

        hsic1 = IndpTest_LKGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all1 = hsic1.perform_test(debug = -1, if_grid_search = True)
        result_test_correct[1, it] = float(results_all1['h0_rejected'])

        hsic2 = IndpTest_LKWeightGaussian(X_tensor, Y_tensor, device, alpha=0.05, n_permutation=100, null_gamma = True, split_ratio = 0.5)
        results_all2 = hsic2.perform_test(debug = -1, if_grid_search = True)
        result_test_correct[2, it] = float(results_all2['h0_rejected'])

        print(alpha, it, np.sum(result_test_correct,1)/(it+1))
    
    result_all_correct.append(result_test_correct)

0.7 0 [1. 1. 1.]
0.7 1 [0.5 0.5 1. ]
0.7 2 [0.33333333 0.33333333 1.        ]
0.7 3 [0.25 0.5  1.  ]
0.7 4 [0.2 0.4 0.8]
0.7 5 [0.16666667 0.33333333 0.83333333]
0.7 6 [0.14285714 0.28571429 0.85714286]
0.7 7 [0.125 0.25  0.875]
0.7 8 [0.11111111 0.22222222 0.88888889]
0.7 9 [0.1 0.3 0.9]
0.7 10 [0.09090909 0.27272727 0.90909091]
0.7 11 [0.08333333 0.25       0.91666667]
0.7 12 [0.07692308 0.30769231 0.92307692]
0.7 13 [0.14285714 0.35714286 0.92857143]
0.7 14 [0.13333333 0.33333333 0.93333333]
0.7 15 [0.125  0.3125 0.875 ]
0.7 16 [0.11764706 0.29411765 0.88235294]
0.7 17 [0.11111111 0.27777778 0.88888889]
0.7 18 [0.10526316 0.31578947 0.89473684]
0.7 19 [0.1 0.3 0.9]
0.7 20 [0.0952381  0.33333333 0.9047619 ]
0.7 21 [0.09090909 0.36363636 0.90909091]
0.7 22 [0.08695652 0.34782609 0.91304348]
0.7 23 [0.08333333 0.375      0.91666667]
0.7 24 [0.08 0.36 0.92]
0.7 25 [0.07692308 0.38461538 0.88461538]
0.7 26 [0.11111111 0.40740741 0.88888889]
0.7 27 [0.10714286 0.42857143 0.89285714]
0.7 2